# Procesamiento de Datos Climatológicos - Estación Augusto Weberbauer (Cajamarca, Perú)

Este notebook procesa un archivo Excel con datos climatológicos registrados por el SENAMHI en la estación **Augusto Weberbauer**, ubicada en Cajamarca, Perú. El periodo de observación abarca desde **1994 hasta 2024**.

## Objetivo

Transformar y limpiar los datos meteorológicos registrados en formato mensual, para generar un dataset diario unificado, adecuado para análisis climáticos, modelado estadístico y aplicaciones de ciencia de datos.

## Variables procesadas

Se procesan las siguientes variables:
- Precipitación diaria
- Temperatura media diaria
- Humedad relativa
- Presión atmosférica
- Velocidad del viento
- Horas de sol

## Etapas del procesamiento

1. Lectura de hojas individuales desde el archivo Excel.
2. Reemplazo de valores:
   - `'T'` (trazas de precipitación) por `0.045`
   - `'S/D'` (sin dato) por `NaN`
3. Conversión de columnas mensuales al formato largo (*tidy*).
4. Unión de todas las variables en un solo DataFrame.
5. Exportación del conjunto limpio para análisis posteriores.

## Repositorio del proyecto

GitHub: [https://github.com/evalcas/climate-variables-senamhi-augusto-weberbauer-cajamarca-1994-2024](https://github.com/evalcas/climate-variables-senamhi-augusto-weberbauer-cajamarca-1994-2024)

## Autor y licencia

- Autor: **Edwin Valencia-Castillo**
- Fecha: **04 de julio de 2025**
- Licencia: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)

## Cita

> Edwin Valencia-Castillo, & Sandra Rodriguez-Avila. (2025). Daily Climate Variables from the SENAMHI Augusto Weberbauer Station (Cajamarca, Peru), 1994–2024 (v1.0.0). Zenodo. https://doi.org/10.5281/zenodo.15803716


In [13]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [15]:
# - Antes de iniciar el proceso se va a convertir en todas las hojas los valores con un valor de T aal valor numerico de 0.045 considerando que según el sensor cuando es T este es <0.1
# - Si su valor es S/D (SIN DATO) asigna un NaN
# - Asigna el nombre de las columnas asi:
# - PRECIPITACION TOTAL DIARIA como PrecipitacionDiaria, TEMPERATURA MEDIA como TemperaturaMedia, HUMEDAD RELATIVA como HumedadRelativa, PRESION ATMOSFERICA como PresionAtmosferica, VELOCIDAD DE VIENTO como VelocidadViento, HORAS DE SOL como HorasSol

def procesar_hoja(df, nombre_columna):

    """
    Preprocesa una hoja de datos climatológicos del archivo Excel original.

    Operaciones realizadas:
    - Reemplaza los valores 'T' por 0.045, asumiendo una traza de precipitación (< 0.1 mm).
    - Reemplaza los valores 'S/D' (sin dato) por NaN.
    - Convierte todas las columnas de meses a valores numéricos (coerción de errores).
    - Reestructura el DataFrame en un formato largo (tidy), con las columnas: YEAR, MES, DIA, <nombre_columna>.

    Parámetros:
    - df (pd.DataFrame): DataFrame original leído desde una hoja del archivo Excel.
    - nombre_columna (str): Nombre final de la variable climatológica (ej., 'PrecipitacionDiaria').

    Retorna:
    - pd.DataFrame: Datos procesados y reestructurados.
    """

    df = df.replace('T', 0.045)
    df = df.replace('S/D', np.nan)


    # Convertir columnas de meses a numéricas, manejando errores
    for col in df.columns[2:]:  # Empezando desde la columna 'ENERO'
      df[col] = pd.to_numeric(df[col], errors='coerce')


    # Renombrar columnas
    columnas_meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SETIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
    meses_num = list(range(1,13))
    meses_dict = dict(zip(columnas_meses, meses_num))

    df_final = pd.DataFrame()
    for mes, num_mes in meses_dict.items():
        temp_df = df[['YEAR', 'DIA', mes]].copy()
        temp_df['MES'] = num_mes
        temp_df = temp_df.rename(columns={mes: nombre_columna})
        df_final = pd.concat([df_final, temp_df], ignore_index=True)

    return df_final


# Leer el archivo Excel
url = 'https://github.com/evalcas/climate-variables-senamhi-augusto-weberbauer-cajamarca-1994-2024/blob/main/dataset_raw_senamhi30years.xlsx?raw=true'
excel_file = url

# Nombre de las hojas en el archivo excel
hojas = ['PRECIPITACION TOTAL DIARIA', 'TEMPERATURA MEDIA', 'HUMEDAD RELATIVA', 'PRESION ATMOSFERICA', 'VELOCIDAD DE VIENTO', 'HORAS DE SOL']
# Nombre de las columnas del dataframe que queremos crear
nombres_columnas = ['PrecipitacionDiaria', 'TemperaturaMedia', 'HumedadRelativa', 'PresionAtmosferica', 'VelocidadViento', 'HorasSol']
# Crear un diccionario para almacenar cada DataFrame
dfs = {}

# Procesar cada hoja y guardar en un diccionario
for hoja, nombre_columna in zip(hojas, nombres_columnas):
  try:
    df = pd.read_excel(excel_file, sheet_name=hoja)
    dfs[nombre_columna] = procesar_hoja(df, nombre_columna)
  except Exception as e:
    print(f"Error procesando la hoja '{hoja}': {e}")


# Fusionar todos los DataFrames
df_final = dfs[nombres_columnas[0]]
for nombre_columna in nombres_columnas[1:]:
  if nombre_columna in dfs: # Verifica si existe
    df_final = pd.merge(df_final, dfs[nombre_columna], on=['YEAR', 'DIA', 'MES'], how='outer')

# Agrupar por año, mes y día
df_final = df_final.groupby(['YEAR', 'MES', 'DIA']).agg({col: 'mean' for col in nombres_columnas}).reset_index()
print(df_final.head())



   YEAR  MES  DIA  PrecipitacionDiaria  TemperaturaMedia  HumedadRelativa  \
0  1994    1    1                  0.2              14.9             76.2   
1  1994    1    2                  7.7              15.1             77.8   
2  1994    1    3                  0.0              14.0             73.1   
3  1994    1    4                  0.0              15.3             66.9   
4  1994    1    5                  5.1              15.1             72.0   

   PresionAtmosferica  VelocidadViento  HorasSol  
0               740.1              1.3       6.8  
1               739.9              1.3       1.6  
2               739.5              1.7       6.8  
3               740.2              NaN       3.5  
4               740.1              2.0       6.2  


In [ ]:
# Hacer una copia del DataFrame final para trabajar sobre ella sin modificar el original
df_completo = df_final.copy()

# Convertir las columnas 'YEAR', 'MES' y 'DIA' a tipo string para facilitar la concatenación posterior
df_completo['YEAR'] = df_completo['YEAR'].astype(str)
df_completo['MES'] = df_completo['MES'].astype(str)
df_completo['DIA'] = df_completo['DIA'].astype(str)

# Crear una nueva columna llamada 'ProcessDate' uniendo las columnas de año, mes y día con el formato 'YYYY-MM-DD'
df_completo['ProcessDate'] = df_completo['YEAR'] + '-' + df_completo['MES'] + '-' + df_completo['DIA']

# Convertir la columna 'ProcessDate' a formato datetime;
# si hay errores en el formato (por ejemplo, fechas inválidas como 2024-02-30), se marcarán como NaT (Not a Time)
# Luego se eliminan las filas con fechas inválidas
try:
    df_completo['ProcessDate'] = pd.to_datetime(df_completo['ProcessDate'], format='%Y-%m-%d', errors='coerce')
    df_completo = df_completo.dropna(subset=['ProcessDate'])
except Exception as e:
    print(f"Error al convertir a datetime: {e}")

# Renombrar las columnas 'YEAR', 'MES' y 'DIA' a 'Year', 'Month' y 'Day' para mejorar la legibilidad y consistencia
df_completo = df_completo.rename(columns={'YEAR': 'Year', 'MES': 'Month', 'DIA': 'Day'})


In [ ]:

# Filtra las filas donde 'ProcessDate' contiene 'nat'
filas_con_nat = df_completo[df_completo['ProcessDate'].astype(str).str.contains('NaT')]
filas_con_nat


,Year,Month,Day,PrecipitacionDiaria,TemperaturaMedia,HumedadRelativa,PresionAtmosferica,VelocidadViento,HorasSol,ProcessDate


In [ ]:
# Renombrar las columnas
# PrecipitacionDiaria -> DailyPrecipitation
# TemperaturaMedia -> AverageTemperature
# HumedadRelativa -> RelativeHumidity
# PresionAtmosferica -> AtmosphericPressure
# VelocidadViento -> WindSpeed
# HorasSol -> SunshineHours

df_completo = df_completo.rename(columns={
    'PrecipitacionDiaria': 'DailyPrecipitation',
    'TemperaturaMedia': 'AverageTemperature',
    'HumedadRelativa': 'RelativeHumidity',
    'PresionAtmosferica': 'AtmosphericPressure',
    'VelocidadViento': 'WindSpeed',
    'HorasSol': 'SunshineHours'
})


print(df_completo.head())

   Year Month Day  DailyPrecipitation  AverageTemperature  RelativeHumidity  \
0  1994     1   1                 0.2                14.9              76.2   
1  1994     1   2                 7.7                15.1              77.8   
2  1994     1   3                 0.0                14.0              73.1   
3  1994     1   4                 0.0                15.3              66.9   
4  1994     1   5                 5.1                15.1              72.0   

   AtmosphericPressure  WindSpeed  SunshineHours ProcessDate  
0                740.1        1.3            6.8  1994-01-01  
1                739.9        1.3            1.6  1994-01-02  
2                739.5        1.7            6.8  1994-01-03  
3                740.2        NaN            3.5  1994-01-04  
4                740.1        2.0            6.2  1994-01-05  


In [ ]:

# Mueve 'ProcessDate' al inicio del DataFrame
cols = list(df_completo.columns)
cols.insert(0, cols.pop(cols.index('ProcessDate')))
df_completo = df_completo.loc[:, cols]

# Visualiza el contenido del DataFrame
df_completo


,ProcessDate,Year,Month,Day,DailyPrecipitation,AverageTemperature,RelativeHumidity,AtmosphericPressure,WindSpeed,SunshineHours
0,1994-01-01,1994,1,1,0.2,14.9,76.2,740.1,1.3,6.8
1,1994-01-02,1994,1,2,7.7,15.1,77.8,739.9,1.3,1.6
2,1994-01-03,1994,1,3,0.0,14.0,73.1,739.5,1.7,6.8
3,1994-01-04,1994,1,4,0.0,15.3,66.9,740.2,NaN,3.5
4,1994-01-05,1994,1,5,5.1,15.1,72.0,740.1,2.0,6.2
...,...,...,...,...,...,...,...,...,...,...
11527,2024-12-27,2024,12,27,11.3,14.0,88.6,740.2,0.3,1.5
11528,2024-12-28,2024,12,28,9.9,15.2,79.8,740.7,0.3,2.3
11529,2024-12-29,2024,12,29,6.6,16.4,71.0,740.4,0.7,1.7
11530,2024-12-30,2024,12,30,0.2,15.8,71.5,739.5,1.3,7.4


In [ ]:

# Encuentra las fechas faltantes
fechas_minima = df_completo['ProcessDate'].min()
fechas_maxima = df_completo['ProcessDate'].max()

rango_fechas = pd.date_range(start=fechas_minima, end=fechas_maxima)
fechas_faltantes = rango_fechas.difference(df_completo['ProcessDate'])

print("Fechas faltantes:")
print(fechas_faltantes)

print("\nFecha más antigua:", fechas_minima)
print("Fecha más reciente:", fechas_maxima)


Fechas faltantes:
DatetimeIndex([], dtype='datetime64[ns]', freq='D')

Fecha más antigua: 1994-01-01 00:00:00
Fecha más reciente: 2024-12-31 00:00:00


In [ ]:

# Muestra las filas donde todas las columnas (excluyendo 'ProcessDate', 'Year', 'Month', 'Day') son NaN
# El dataframe contiene los datos procesados sin embargo hay datos faltantes producto de la pandemia u otros eventos que no fueron recogidos y es necesario completarlos a traves de metodos de imputación
filas_todos_nan = df_completo[df_completo.drop(columns=['ProcessDate', 'Year', 'Month', 'Day']).isna().all(axis=1)]
filas_todos_nan


,ProcessDate,Year,Month,Day,DailyPrecipitation,AverageTemperature,RelativeHumidity,AtmosphericPressure,WindSpeed,SunshineHours
5549,2008-12-01,2008,12,1,NaN,NaN,NaN,NaN,NaN,NaN
5550,2008-12-02,2008,12,2,NaN,NaN,NaN,NaN,NaN,NaN
5551,2008-12-03,2008,12,3,NaN,NaN,NaN,NaN,NaN,NaN
5552,2008-12-04,2008,12,4,NaN,NaN,NaN,NaN,NaN,NaN
5553,2008-12-05,2008,12,5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9852,2020-06-26,2020,6,26,NaN,NaN,NaN,NaN,NaN,NaN
9853,2020-06-27,2020,6,27,NaN,NaN,NaN,NaN,NaN,NaN
9854,2020-06-28,2020,6,28,NaN,NaN,NaN,NaN,NaN,NaN
9855,2020-06-29,2020,6,29,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# Save the DataFrame to an Excel file in Google Drive
df_completo.to_excel('/content/drive/My Drive/Colab Notebooks/Research/DataSENAMHI30Year/dataset_preprocessed_senamhi30Year.xlsx', index=False)


In [ ]:
# Leyendo los datos ya preprocesados
df_procesado = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Research/DataSENAMHI30Year/dataset_preprocessed_senamhi30Year.xlsx')


In [ ]:
# verifica que no haya fechas faltantes, si los hay visualizalos
# luego muestra los datos faltantes del df_procesado

# Establecer 'ProcessDate' como índice
df_procesado = df_procesado.set_index('ProcessDate')

# Verificar fechas faltantes
fechas_minima = df_procesado.index.min()
fechas_maxima = df_procesado.index.max()

rango_fechas = pd.date_range(start=fechas_minima, end=fechas_maxima)
fechas_faltantes = rango_fechas.difference(df_procesado.index)

if not fechas_faltantes.empty:
  print("Fechas faltantes:")
  print(fechas_faltantes)
else:
  print("No hay fechas faltantes en el DataFrame.")

# Mostrar datos faltantes en el DataFrame
print("\nDatos faltantes en df_procesado:")
print(df_procesado.isna().sum())


No hay fechas faltantes en el DataFrame.

Datos faltantes en df_procesado:
Year                      0
Month                     0
Day                       0
DailyPrecipitation      137
AverageTemperature      396
RelativeHumidity        508
AtmosphericPressure     327
WindSpeed              1288
SunshineHours           163
dtype: int64


In [ ]:
# Datos del dataframe
df_procesado.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11323 entries, 1994-01-01 to 2024-12-31
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 11323 non-null  int64  
 1   Month                11323 non-null  int64  
 2   Day                  11323 non-null  int64  
 3   DailyPrecipitation   11186 non-null  float64
 4   AverageTemperature   10927 non-null  float64
 5   RelativeHumidity     10815 non-null  float64
 6   AtmosphericPressure  10996 non-null  float64
 7   WindSpeed            10035 non-null  float64
 8   SunshineHours        11160 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 884.6 KB
